# CCSS Web Scraping in Python (BeautifulSoup)

First. click 'File' -> 'Save a copy in Drive' to work in your own session.

The first module will introduce the fundamentals of web scraping in Python by highlighting the Beautiful Soup library. The second module will explore how to navigate dynamically generated website content through interactive scraping powered by Selenium.

We'll be using WebScraper.io's [webscraping test site](https://webscraper.io/test-sites/e-commerce/static) structured as an electronics e-commerce site throughout the exercises. This webpage serves as a great testing ground to learn about web scraping without having to worry about site blockages when building our initial scraper. We'll be toggling back and forth between this notebook and the website to use its Inspect feature to learn more about the site's underlying structure throughout the demos.


#Module 1- Fundamentals with Beautiful Soup

Let's start by importing in the libraries we'll be using within the first module:

In [ ]:
!pip install bs4
!pip install pandas
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd
import pprint


Our initial goal will be to retrieve the name, price, description, and number of reviews of the three electronics featured on the site home page. `urllib.requests` is a library that goes hand-in-hand with Beautiful Soup by establishing the HTTP connection we'll need with the e-commerce site to parse the site's HTML. Let's start by inspecting the full HTML of the site itself by directly using the `BeautifulSoup` method with a designated `html.parser`:

In [ ]:
html = urlopen('https://webscraper.io/test-sites/e-commerce/static')
bs_html = BeautifulSoup(html, 'html.parser')
print(bs_html)

<!DOCTYPE html>

<html lang="en">
<head>
<!-- Google Tag Manager -->
<script nonce="24vpS8SCYBLiwyojAr8PtGe67Er1dFxn">(function (w, d, s, l, i) {
		w[l] = w[l] || [];
		w[l].push({
			'gtm.start':
				new Date().getTime(), event: 'gtm.js'
		});
		var f = d.getElementsByTagName(s)[0],
			j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : '';
		j.async = true;
		j.src =
			'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
		f.parentNode.insertBefore(j, f);
	})(window, document, 'script', 'dataLayer', 'GTM-NVFPDWB');</script>
<!-- End Google Tag Manager -->
<title>Static | Web Scraper Test Sites</title>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="web scraping,Web Scraper,Chrome extension,Crawling,Cross platform scraper" name="keywords">
<meta content="Test Web Scraper's features and performance on mock e-commerce sites. Extract product data, prices, and categories in a controlled environment." name="description">
<l

### HTML Tags & Attributes

- **Tags** are used to represent different component types within an HTML file, such as the `<title>`, `<div>` for a designated section within the document, and `<h1>` for the first header.
- HTML tags usually require a closing tag such as </title> or </div>. Sections in HTML begin and end.
- HTML tags will often include additional information within the tag itself that are known as **attributes**. You can think of attributes as an additional identifier for a given tag.
- Attributes are marked by key words such as `class`, `id`, or `href` (referring to hyperlinks) followed by an equal sign and a label such as `<div class='container'>`

Here are two external references for learning more about the available [tags](https://www.w3schools.com/tags/default.asp) and [attributes](https://www.w3schools.com/tags/ref_attributes.asp) within HTML.


Our created Beautiful Soup parsed HTML object offers a variety of methods that allows us to look at specific tags within the HTML. The following retrieves the first specified instance of the `h4` heading tag in the HTML file. Then finds all instances of `h4` heading tag in the HTML file:

In [ ]:
#find option pulls the first occurence.
name = bs_html.find('h4')
print(name)
print(name.get_text())

#find_all includes all tags.
names = bs_html.find_all('h4')
print(names)

<h4 class="price float-end card-title pull-right">$1096.02</h4>
$1096.02
[<h4 class="price float-end card-title pull-right">$1096.02</h4>, <h4>
<a class="title" href="/test-sites/e-commerce/static/product/94" title="Lenovo ThinkPad L460">Lenovo ThinkPa...</a>
</h4>, <h4 class="price float-end card-title pull-right">$1299</h4>, <h4>
<a class="title" href="/test-sites/e-commerce/static/product/135" title="MSI GL62VR 7RFX">MSI GL62VR 7RF...</a>
</h4>, <h4 class="price float-end card-title pull-right">$306.99</h4>, <h4>
<a class="title" href="/test-sites/e-commerce/static/product/32" title="Aspire E1-510">Aspire E1-510</a>
</h4>]


Attributes are used to distinguish different subgroups of the same base HTML tag. This facilitates the easy retrieval of distinct HTML elements within a web scraping program that we would otherwise struggle to differentiate due to having the same tags. An example of this is the e-commerce test site `<div class="caption">` and `<div class="ratings">` to distinguish between the product descriptions and the ratings of the items being displayed on the website.

We can use the `find_all` method to identify any of the `div` tags with their `class` attribute specified as a `caption` as follows:

In [ ]:
bs_html.find_all('div', {'class':'caption'})

[<div class="caption">
 <h4 class="price float-end card-title pull-right">$1096.02</h4>
 <h4>
 <a class="title" href="/test-sites/e-commerce/static/product/94" title="Lenovo ThinkPad L460">Lenovo ThinkPa...</a>
 </h4>
 <p class="description card-text">Lenovo ThinkPad L460, 14" FHD IPS, Core i7-6600U, 8GB, 256GB SSD, Windows 10 Pro</p>
 </div>,
 <div class="caption">
 <h4 class="price float-end card-title pull-right">$1299</h4>
 <h4>
 <a class="title" href="/test-sites/e-commerce/static/product/135" title="MSI GL62VR 7RFX">MSI GL62VR 7RF...</a>
 </h4>
 <p class="description card-text">MSI GL62VR 7RFX, 15.6" FHD, Core i7-7700HQ, 8GB, 1TB + 256GB SSD, GeForce GTX 1060 3GB, Windows 10 Home</p>
 </div>,
 <div class="caption">
 <h4 class="price float-end card-title pull-right">$306.99</h4>
 <h4>
 <a class="title" href="/test-sites/e-commerce/static/product/32" title="Aspire E1-510">Aspire E1-510</a>
 </h4>
 <p class="description card-text">15.6", Pentium N3520 2.16GHz, 4GB, 500GB, Linux</p>


The `get_text` method strips away the HTML tags to just express the text itself:

In [ ]:
captions = bs_html.find_all('div', {'class':'caption'})
for name in captions:
    print(name.get_text())


$1096.02

Lenovo ThinkPa...

Lenovo ThinkPad L460, 14" FHD IPS, Core i7-6600U, 8GB, 256GB SSD, Windows 10 Pro


$1299

MSI GL62VR 7RF...

MSI GL62VR 7RFX, 15.6" FHD, Core i7-7700HQ, 8GB, 1TB + 256GB SSD, GeForce GTX 1060 3GB, Windows 10 Home


$306.99

Aspire E1-510

15.6", Pentium N3520 2.16GHz, 4GB, 500GB, Linux



The basic workflow behind using Beautiful Soups' `find` and `find_all` methods is to first specify the tag you're interested in within the HTML document, and then the associated attributes that further narrows your search from the tag.

We can build from this to retrieve multiple tag types at once, such as all of the headers set to the 1, 2, and 4 subheader sizes as follows:  

In [ ]:
bs_html.find_all(['h1','h2','h4'])

[<h1>Test Sites</h1>,
 <h2>E-commerce training site</h2>,
 <h4 class="price float-end card-title pull-right">$1096.02</h4>,
 <h4>
 <a class="title" href="/test-sites/e-commerce/static/product/94" title="Lenovo ThinkPad L460">Lenovo ThinkPa...</a>
 </h4>,
 <h4 class="price float-end card-title pull-right">$1299</h4>,
 <h4>
 <a class="title" href="/test-sites/e-commerce/static/product/135" title="MSI GL62VR 7RFX">MSI GL62VR 7RF...</a>
 </h4>,
 <h4 class="price float-end card-title pull-right">$306.99</h4>,
 <h4>
 <a class="title" href="/test-sites/e-commerce/static/product/32" title="Aspire E1-510">Aspire E1-510</a>
 </h4>]

We can also identify multiple attribute matches to the same tag in a single `find_all` call. We achieve this by setting our second parameter as a dictionary with the `class` attribute as the key as the attribute labels `caption` and `ratings` as values. Another way to extract what we did above.

In [ ]:
caption_reviews = bs_html.find_all('div', {'class':{'caption', 'ratings'}})
for name in caption_reviews:
    print(name.get_text())


$1096.02

Lenovo ThinkPa...

Lenovo ThinkPad L460, 14" FHD IPS, Core i7-6600U, 8GB, 256GB SSD, Windows 10 Pro


14 reviews








$1299

MSI GL62VR 7RF...

MSI GL62VR 7RFX, 15.6" FHD, Core i7-7700HQ, 8GB, 1TB + 256GB SSD, GeForce GTX 1060 3GB, Windows 10 Home


1 reviews







$306.99

Aspire E1-510

15.6", Pentium N3520 2.16GHz, 4GB, 500GB, Linux


2 reviews








If you look closely, you'll notice how the values for the electronic goods returned from Beautiful Soup don't actually align with what you currently see when looking at the site directly via Inspect. The site is updating the items shown on the homepage outside of default values set within the HTML.

### Saving Scraped Data

After identifying the tags and attributes that will allow us to match with the specific data we're interested in collecting from the website's HTML, we'd then proceed with the actual data collection within our scraper. The `pandas` library naturally intergrates with Beautiful Soup HTML parsing by structuring different tag retrievals into designated data columns.

Let's build up to creating a function that retrieves the product data we're interested in via the e-commerce site's URL and stores it within a pandas DataFrame for us. We can envision creating columns that separate out the title, description, price, and reviews of each of the products on the site's home page.

(Check out [CCSS' workshop on pandas](https://github.com/ccss-rs/python-bootcamp/blob/main/pandas.ipynb) on your own if you haven't worked with Pandas before or need a refresher.)  

A helpful strategy that moves us closer to our goal of producing a final data frame is using a different Beautiful Soup pattern matching of the HTML tag that has the data we're interested in with the tag's unique attributes and passing the text of all matches into a list through list comprehension statements.

Here's an example for the `a` tag distinguished by the `class` attribute labeled as `title`:

As we're building up to a function that will represent our web scraped data into a row and column format that will be ideal for final data storage on our local computer, it's often helpful to think through what data types can be handled by pandas that effectively matches column names (title, description, price, and reviews) to rows representing each scraped product.

A go-to is dictionaries since the key-value structure naturally translates to a column-row format within pandas. We'll therefore create a dictionary to hold our scraped data from Beautiful Soup as follows:

In [ ]:
caption_reviews = bs_html.find_all('div', {'class':{'caption', 'ratings'}})
print(caption_reviews)

[<div class="caption">
<h4 class="price float-end card-title pull-right">$1281.99</h4>
<h4>
<a class="title" href="/test-sites/e-commerce/static/product/40" title="Dell XPS 13">Dell XPS 13</a>
</h4>
<p class="description card-text">13.3" Touch, Core i5-4210U, 8GB, 128GB SSD, Windows 8.1</p>
</div>, <div class="ratings">
<p class="review-count float-end">4 reviews</p>
<p data-rating="3">
<span class="ws-icon ws-icon-star"></span>
<span class="ws-icon ws-icon-star"></span>
<span class="ws-icon ws-icon-star"></span>
</p>
</div>, <div class="caption">
<h4 class="price float-end card-title pull-right">$1102.66</h4>
<h4>
<a class="title" href="/test-sites/e-commerce/static/product/99" title="Dell Latitude 5280">Dell Latitude...</a>
</h4>
<p class="description card-text">Dell Latitude 5280, 12.5" HD, Core i5-7300U, 8GB, 256GB SSD, Windows 10 Pro</p>
</div>, <div class="ratings">
<p class="review-count float-end">8 reviews</p>
<p data-rating="1">
<span class="ws-icon ws-icon-star"></span>
</p>

In [ ]:
title = [values.get_text() for values in bs_html.find_all('a', {'class': 'title'})]
links = [values['href'] for values in bs_html.find_all('a', {'class': 'title'})]
desc = [values.get_text() for values in bs_html.find_all('p', {'class': 'description'})]
price = [values.get_text() for values in bs_html.find_all('h4', {'class': 'price float-end card-title pull-right'})]
reviews = [values.get_text() for values in bs_html.find_all('p', {'class': 'description card-text'})]

#Create pandas dataframe.
data = pd.DataFrame({
    'title': title,
    'price': price,
    'url': links,
    'description': desc,
    'reviews': reviews
})
display(data)

,title,price,url,description,reviews
0,Dell XPS 13,$1281.99,/test-sites/e-commerce/static/product/40,"13.3"" Touch, Core i5-4210U, 8GB, 128GB SSD, Wi...","13.3"" Touch, Core i5-4210U, 8GB, 128GB SSD, Wi..."
1,Dell Latitude...,$1102.66,/test-sites/e-commerce/static/product/99,"Dell Latitude 5280, 12.5"" HD, Core i5-7300U, 8...","Dell Latitude 5280, 12.5"" HD, Core i5-7300U, 8..."
2,Lenovo V510 Bl...,$487.8,/test-sites/e-commerce/static/product/89,"Lenovo V510 Black, 15.6"" HD, Core i3-6006U, 4G...","Lenovo V510 Black, 15.6"" HD, Core i3-6006U, 4G..."


Now that we have the individual components thought through regarding how to automate collecting data from the website in a format ideal to create a final pandas data frame with, we're ready to combine these steps into one function that will run through the entire process for us and return the data frame as its output.

`html_to_pandas` achieves this for us:

In [ ]:
def extract(url):
    # Establish our connection to the website and create a Beautiful Soup object
    html = urlopen(url)
    bs_html = BeautifulSoup(html, 'html.parser')
    #Extract desired contents
    title = [values.get_text() for values in bs_html.find_all('a', {'class': 'title'})]
    links = [values['href'] for values in bs_html.find_all('a', {'class': 'title'})]
    desc = [values.get_text() for values in bs_html.find_all('p', {'class': 'description'})]
    price = [values.get_text() for values in bs_html.find_all('h4', {'class': 'price float-end card-title pull-right'})]
    reviews = [values.get_text() for values in bs_html.find_all('p', {'class': 'description card-text'})]

    #Place into data frame
    data = pd.DataFrame({
    'title': title,
    'price': price,
    'url': links,
    'description': desc,
    'reviews': reviews
    })

    #Output dataframe
    return data

In [ ]:
data = extract('https://webscraper.io/test-sites/e-commerce/static')
display(data)

,title,price,url,description,reviews
0,Dell XPS 13,$1281.99,/test-sites/e-commerce/static/product/40,"13.3"" Touch, Core i5-4210U, 8GB, 128GB SSD, Wi...","13.3"" Touch, Core i5-4210U, 8GB, 128GB SSD, Wi..."
1,Dell Latitude...,$1102.66,/test-sites/e-commerce/static/product/99,"Dell Latitude 5280, 12.5"" HD, Core i5-7300U, 8...","Dell Latitude 5280, 12.5"" HD, Core i5-7300U, 8..."
2,Lenovo V510 Bl...,$487.8,/test-sites/e-commerce/static/product/89,"Lenovo V510 Black, 15.6"" HD, Core i3-6006U, 4G...","Lenovo V510 Black, 15.6"" HD, Core i3-6006U, 4G..."


We've successfully gone from a whole jumble of HTML and website text to a ordered DataFrame with the exact information we're interested in via Beautiful Soup. In an actual web scraping data collection workflow you'd likely save the above data frame to your personal computer such as via a CSV file.